References:
* Kaggle Official: https://www.kaggle.com/dansbecker/permutation-importance
* sarmat https://www.kaggle.com/sarmat/lgbm-permutation-importance

In [ ]:
import seaborn as sns
import random
import pprint as pp
import gc
import os
import time
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from scipy import stats
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold,StratifiedKFold, RepeatedKFold
from operator import itemgetter
import os
import time
import datetime
import warnings
warnings.filterwarnings('ignore')
print(os.listdir("../input/ieee-base-data-pkl/"))

import gc

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def missing_values_table(df):# Function to calculate missing values by column# Funct 
    mis_val = df.isnull().sum() # Total missing values
    mis_val_pct = 100 * df.isnull().sum() / len(df)# Percentage of missing values
    mis_val_df = pd.concat([mis_val, mis_val_pct], axis=1)# Make a table with the results
    mis_val_df_cols = mis_val_df.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})# Rename the columns
    mis_val_df_cols = mis_val_df_cols[mis_val_df_cols.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)# Sort the table by percentage of missing descending
    print ("Dataframe has " + str(df.shape[1]) + " columns.\n" 
           "There are " + str(mis_val_df_cols.shape[0]) + " cols having missing values.")# Print some summary information
    return mis_val_df_cols # Return the dataframe with missing information

def Negativedownsampling(train, ratio) :
    

    # Number of data points in the minority class
    number_records_fraud = len(train[train.isFraud == 1])
    fraud_indices = np.array(train[train.isFraud == 1].index)

    # Picking the indices of the normal classes
    normal_indices = train[train.isFraud == 0].index

    # Out of the indices we picked, randomly select "x" number (number_records_fraud)
    random_normal_indices = np.random.choice(normal_indices, number_records_fraud*ratio, replace = False)
    random_normal_indices = np.array(random_normal_indices)

    # Appending the 2 indices
    under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

    # Under sample dataset
    under_sample_data = train.iloc[under_sample_indices,:]
    
    # Showing ratio
    print("Percentage of normal transactions: ", round(len(under_sample_data[under_sample_data.isFraud == 0])/len(under_sample_data),2)* 100,"%")
    print("Percentage of fraud transactions: ", round(len(under_sample_data[under_sample_data.isFraud == 1])/len(under_sample_data),2)* 100,"%")
    print("Total number of transactions in resampled data: ", len(under_sample_data))
    
    return under_sample_data

In [ ]:
train_df = reduce_mem_usage(pd.read_pickle('../input/ieee-base-data-pkl/train_transaction.pkl'))
test_df = reduce_mem_usage(pd.read_pickle('../input/ieee-base-data-pkl/test_transaction.pkl'))

# NegativeDownSample

discussion: https://www.kaggle.com/c/ieee-fraud-detection/discussion/108616#latest-634925

kernel: https://www.kaggle.com/zero92/negative-downsampling-improve-the-score

In [ ]:
train_df = Negativedownsampling(train_df, 1)

In [ ]:
print(train_df.shape, test_df.shape)

In [ ]:
train_Y = train_df['isFraud']

train_df.drop(['isFraud','TransactionID'],1,inplace=True)
test_df.drop(['TransactionID'],1,inplace=True)

In [ ]:
train_X = train_df
test_X = test_df
del train_df; gc.collect()

In [ ]:
import eli5
from IPython.display import display
from eli5.permutation_importance import get_score_importances
from eli5.sklearn import PermutationImportance

In [ ]:
# TO DROP NAN for permutation

cate_drop = train_X.loc[:,train_X.dtypes=='category'].columns.tolist()

print(train_X.shape)
train_X.drop(cate_drop,1,inplace=True)
test_X.drop(cate_drop,1,inplace=True)
train_X.fillna(0, inplace=True)
test_X.fillna(0, inplace=True)
print(train_X.shape)

In [ ]:
gc.collect()

# MODEL

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

oof_lgb = np.zeros(len(train_X))
predictions = np.zeros(len(test_X))
feature_importance_df = pd.DataFrame()
permu_importance_df = pd.DataFrame()
cv_score_df = []

# Model parameters
lgb_params = {'num_leaves': 2**8,
             #'min_data_in_leaf': 10, 
             'objective':'binary',
             'max_depth': -1,
             'learning_rate': 0.5,
             "boosting": "gbdt",
             "feature_fraction": 1,
             "bagging_freq": 1,
             "bagging_fraction": 1,
             "bagging_seed": 42,
             "metric": 'auc',
             "lambda_l1": 0.0,
             "verbosity": 100,
             "nthread": -1,
             "random_state": 42}

model_start = time.time()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X)):
    strLog = "fold {}".format(fold_)
    print(strLog+ "-" * 50)

    X_tr, X_val = train_X.iloc[trn_idx], train_X.iloc[val_idx]
    y_tr, y_val = train_Y.iloc[trn_idx], train_Y.iloc[val_idx]

    model = lgb.LGBMClassifier(**lgb_params, n_estimators = 200000, n_jobs = -1)
    model.fit(X_tr, 
              y_tr, 
              eval_set=[(X_tr, y_tr), (X_val, y_val)], 
              eval_metric='auc',
              verbose=10000, 
              early_stopping_rounds=100)
    oof_lgb[val_idx] = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:,1]

    cv_score_df.append(model.best_score_)

    #feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_X.columns
    fold_importance_df["importance"] = model.feature_importances_[:len(train_X.columns)]
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    ####### Permutation Importance #######
    perm = PermutationImportance(model,scoring=None, n_iter=1, random_state=42, cv=None, refit=False).fit(X_val, y_val)
    tmp = eli5.show_weights(perm)
    display(eli5.show_weights(perm, top = 10, feature_names = list(train_X.columns)))
    
    fold_permu_df = eli5.explain_weights_df(perm, top = len(list(train_X.columns)), feature_names = list(train_X.columns))
    fold_permu_df["fold"] = fold_ + 1
    permu_importance_df = pd.concat([permu_importance_df, fold_permu_df], axis=0)
    ######## predictions
    predictions += model.predict_proba(test_X, num_iteration=model.best_iteration_)[:,1] / folds.n_splits

cv_score_df = pd.DataFrame.from_dict(cv_score_df)
cv_score_df = cv_score_df.valid_1.tolist()
cv_score_df = list(map(itemgetter('auc'),cv_score_df))

print("-" * 50)
#print("SF   RMSE = {}".format(oof_score))
print("Mean AUC" , " = {}".format(np.mean(cv_score_df)))
print("Std AUC" , " = {}".format(np.std(cv_score_df)))
lgb.plot_metric(model, metric='auc', title='auc plot', xlabel='Iterations', ylabel='auto', figsize=(10,8), grid=False)

model_end = time.time()
model_elapsed = model_end - model_start
print('Model elapsed {0:0.2f}'.format(model_elapsed/60), "minutes.")
cols = (feature_importance_df[["Feature", "importance"]]
    .groupby("Feature")
    .mean()
    .sort_values(by="importance", ascending=False)[:1000].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]
pd.set_option('display.max_rows', 500)
best_features = best_features.drop(['fold'],axis=1)
best_features = best_features.groupby(['Feature'], as_index = False).mean()
best_features['Feature Rank'] = best_features['importance'].rank(ascending=0)
best_features = best_features.sort_values('Feature Rank', ascending = True)
print("Best Feature")
pp.pprint(best_features.loc[best_features['importance']!=0].head(10))

permu_features = permu_importance_df.loc[permu_importance_df.feature.isin(cols)]
permu_features = permu_features.drop(['fold'],axis=1)
permu_features = permu_features.groupby(['feature'], as_index=False).mean()
permu_features['Feature Rank'] = permu_features['weight'].rank(ascending=0)
permu_features = permu_features.sort_values('Feature Rank', ascending=True)
print("Permutation Feature")
print(permu_features.head(10))

# RESULT

In [ ]:
best_features.head()

In [ ]:
permu_features.head()

In [ ]:
best_features.to_csv("best_features.csv", index=False)
permu_features.to_csv("permu_features.csv", index=False)